In [47]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
import numpy as np
import pandas as pd

#fastai
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *
from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import * 
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

#pytorch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V


In [48]:
# set theme 
# from jupyterthemes.stylefx import set_nb_theme
# set_nb_theme('monokai')

In [49]:
PATH=Path('data/hemingway')
PATH.mkdir(exist_ok=True)

In [50]:
str(PATH)

'data/hemingway'

## save full hemingway with code

In [51]:
file = open(Path('/notebooks/volume/full_hemingway.txt'), encoding='utf-8')
fulltext = file.read()

In [52]:
r = np.random.randint(1,len(fulltext))
fulltext[r:(r+300)]

' have been about sliding. But listen, Billy, and I’ll\ntell you a secret. Stick to sheets, Billy. Keep away from women and\nhorses and, and—” he stopped “—eagles, Billy. If you love horses\nyou’ll get horse-s—, and if you love eagles you’ll get eagle-s—.” He\nstopped and put his head under the sheet.\n\n“'

In [68]:
LanguageModelData?

### split hemingway by simply by line

In [21]:
lng = len(fulltext)
trn,val,test = [fulltext[0:int(lng*i)] for i in [0.7,0.2,0.1]]

In [57]:
# save to files
def save_txt_file(name, txt):    
    f=open(PATH/name, mode='w', encoding='utf-8')
    f.write(txt)
    f.close()

In [59]:
save_txt_file('hemingway.trn.txt', trn)
save_txt_file('hemingway.val.txt', val)
save_txt_file('hemingway.text.txt', test)

# Data loader 

In [71]:
bs, bptt = 80, 70
TEXT = data.Field(lower=True, tokenize='spacy')
FILES = dict(train='hemingway.trn.txt', val='hemingway.val.txt', test='hemingway.text.txt')
md = LanguageModelData(PATH, TEXT, *FILES, bs=bs, bptt=bptt, min_freq=10)

OSError: Can't find model 'en'

In [66]:
data.Field??

In [63]:
spacy??